In [ ]:
# Required Libraries
!pip install yt-dlp torch torchvision torchaudio transformers
# Install required libraries
!pip install yt-dlp
!pip install git+https://github.com/openai/whisper.git
!pip install torch transformers
!apt-get install ffmpeg

ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.9 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ys5n9l13
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ys5n9l13
  Resolved https://github.com/openai/whisper.git to commit 25639fc17ddc013d56c594bfbf7644f2185fad84
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 

In [ ]:
import yt_dlp
import whisper
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration

# Function to download audio from YouTube using yt-dlp
def download_audio(youtube_url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': 'audio.%(ext)s',
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([youtube_url])
        return 'audio.mp3'  # The output file
    except Exception as e:
        print(f"Error downloading audio: {e}")
        return None

# Function to transcribe audio using Whisper
def transcribe_audio(audio_file):
    model = whisper.load_model("base")  # Choose appropriate model size
    result = model.transcribe(audio_file)
    return result['text']

# Function to split text into chunks for summarization
def split_text(text, chunk_size=512):
    sentences = text.split('. ')
    chunks = []
    current_chunk = []

    for sentence in sentences:
        current_chunk.append(sentence)
        if len(' '.join(current_chunk)) > chunk_size:
            chunks.append(' '.join(current_chunk))
            current_chunk = []

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Function to summarize text using T5-base
def summarize_text_with_t5(text):
    # Load T5 model and tokenizer
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    tokenizer = T5Tokenizer.from_pretrained("t5-base")

    chunks = split_text(text)
    summaries = []

    for chunk in chunks:
        input_ids = tokenizer.encode("summarize: " + chunk, return_tensors="pt", max_length=512, truncation=True)
        summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    return ' '.join(summaries)

# Main function to summarize a YouTube video
def summarize_youtube_video(youtube_url):
    audio_file = download_audio(youtube_url)
    if audio_file is None:
        return "Failed to download audio."

    transcript = transcribe_audio(audio_file)
    summary = summarize_text_with_t5(transcript)
    return summary

# Example usage
youtube_url = 'https://youtu.be/fRJQ-I0hArU?si=KUd_U_bt76qw_10q'  # Replace with your YouTube video ID
summary = summarize_youtube_video(youtube_url)
print("Summary:", summary)


[youtube] Extracting URL: https://youtu.be/fRJQ-I0hArU?si=KUd_U_bt76qw_10q
[youtube] fRJQ-I0hArU: Downloading webpage
[youtube] fRJQ-I0hArU: Downloading ios player API JSON
[youtube] fRJQ-I0hArU: Downloading mweb player API JSON
[youtube] fRJQ-I0hArU: Downloading player 2f238d39
[youtube] fRJQ-I0hArU: Downloading m3u8 information
[info] fRJQ-I0hArU: Downloading 1 format(s): 251
[download] Destination: audio.webm
[download] 100% of    8.23MiB in 00:00:00 at 20.34MiB/s  
[ExtractAudio] Destination: audio.mp3
Deleting original file audio.webm (pass -k to keep)


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 86.8MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Summary: the new iPhone 16 Pro Max will be my last week with it to make that full 30 days use. on paper and looks wise, the big change is obviously the new color option. the big change is obviously the titanium railings around the outside. the back is still glass We have a new addition of a camera control button. the phone is grown by a couple millimeters and it just has a larger screen to body ratio. if you are rocking in case, you'll need something new. this little camera guard actually protects the lenses. you can use it as a little kick flip stand to watch content both horizontally and vertically. they also have a hybrid case as well as their tough armor. the new iphone has a three layer design that can withstand drops from 23 feet, whereas the classic hybrid and cloud soft cases withstand drops up to 11 feet. the screen is still soft and within 30 seconds from unboxing this, I actually scratched my new phone. the new a18 pro has a 6-core CPU, 6-core GPU. it's coming later in the f